In [8]:
#------------------------------- Tradiction --------------------------#
!pip install googletrans==3.1.0a0
import pandas as pd
from googletrans import Translator

# Lire le fichier CSV
df = pd.read_csv('/content/email.csv')

# Initialiser le traducteur
translator = Translator()

# Fonction pour traduire un message
def translate_message(message, dest_lang='mg'):
    try:
        translation = translator.translate(message, dest=dest_lang)
        return translation.text
    except Exception as e:
        print(f"Error translating message: {e}")
        return message  # Retourne le message original en cas d'erreur

# Traduire les messages de la colonne 'Message'
df['Message Traduit'] = df['Message'].apply(lambda x: translate_message(x, dest_lang='mg'))

# Sauvegarder le fichier avec les messages traduits
df.to_csv('/content/email_traduit.csv', index=False)

print("Traduction terminée. Le fichier traduit a été sauvegardé.")

Traduction terminée. Le fichier traduit a été sauvegardé.


In [7]:
# Install the necessary packages
!pip install googletrans deep-translator
import pandas as pd
from deep_translator import GoogleTranslator

# Lire le fichier CSV
df = pd.read_csv('/content/email_traduit.csv')

# Initialiser le traducteur
translator = GoogleTranslator(source='auto', target='mg')

# Fonction pour traduire un message
def translate_message(message):
    try:
        return translator.translate(message)
    except Exception as e:
        print(f"Error translating message: {e}")
        return message  # Retourne le message original en cas d'erreur

# Traduire les messages de la colonne 'Message'
df['Message Traduit'] = df['Message'].apply(translate_message)

# Sauvegarder le fichier avec les messages traduits
df.to_csv('/content/email_traduit_1.csv', index=False)

print("Traduction terminée. Le fichier traduit a été sauvegardé.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 649.3 kB/s eta 0:00:00
Traduction terminée. Le fichier traduit a été sauvegardé.


In [5]:
#------------------------------- RandomForestClassifier --------------------------#
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux, les chiffres, et convertir en minuscules
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    return text.lower()

# Lire le fichier CSV avec les messages traduits
df = pd.read_csv('/content/email_traduit.csv')

# Nettoyage des données
df['Message Traduit'] = df['Message Traduit'].apply(clean_text)

# Préparer les données pour l'entraînement
X = df['Message Traduit']
y = df['Category']  # Remplacez 'Label' par le nom de la colonne qui contient les étiquettes

# Vectorisation des textes
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Entraînement du modèle RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)

# Test du modèle
y_pred_rf = clf_rf.predict(X_test)

# Évaluation des performances du modèle
print("Accuracy (RandomForest):", accuracy_score(y_test, y_pred_rf))
print("Classification Report (RandomForest):\n", classification_report(y_test, y_pred_rf))

# Sauvegarde du modèle entraîné avec joblib
model_filename_rf = '/content/sample_data/random_forest_model.joblib'
joblib.dump(clf_rf, model_filename_rf)
print(f"Modèle RandomForest sauvegardé sous le nom {model_filename_rf}")

# Sauvegarde du vectorizer
vectorizer_filename_rf = '/content/sample_data/tfidf_vectorizer_random.joblib'
joblib.dump(vectorizer, vectorizer_filename_rf)
print(f"Vectorizer RandomForest sauvegardé sous le nom {vectorizer_filename_rf}")

# Charger le modèle et le vectorizer pour faire des prédictions
loaded_model_rf = joblib.load(model_filename_rf)
loaded_vectorizer_rf = joblib.load(vectorizer_filename_rf)

# Exemple de prédiction sur un nouveau message
new_message = input("Entrer votre message: ")
new_message_tfidf = loaded_vectorizer_rf.transform([clean_text(new_message)])  # Passer le message sous forme de liste
prediction_rf = loaded_model_rf.predict(new_message_tfidf)

print(f"Prédiction pour le nouveau message (RandomForest): {prediction_rf[0]}")


Accuracy (RandomForest): 0.9730941704035875
Classification Report (RandomForest):
               precision    recall  f1-score   support

         ham       0.97      1.00      0.98       958
        spam       1.00      0.81      0.89       157

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Modèle RandomForest sauvegardé sous le nom /content/sample_data/random_forest_model.joblib
Vectorizer RandomForest sauvegardé sous le nom /content/sample_data/tfidf_vectorizer_random.joblib
Entrer votre message: salama tsara daholo ve ianareo
Prédiction pour le nouveau message (RandomForest): ham


In [6]:
#---------------------------- Regression Logistique -------------------------------#
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux, les chiffres, et convertir en minuscules
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    return text.lower()

# Lire le fichier CSV avec les messages traduits
df = pd.read_csv('/content/email_traduit.csv')

# Nettoyage des données
df['Message Traduit'] = df['Message Traduit'].apply(clean_text)

# Préparation des données
X = df['Message Traduit']  # Les textes traduits nettoyés
y = df['Category']  # Les étiquettes (spam ou ham)

# Vectorisation des textes en utilisant TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Entraînement du modèle LogisticRegression
clf_lr = LogisticRegression(random_state=42)
clf_lr.fit(X_train, y_train)

# Test du modèle
y_pred_lr = clf_lr.predict(X_test)

# Évaluation des performances du modèle
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred_lr))
print("Classification Report (Logistic Regression):\n", classification_report(y_test, y_pred_lr))

# Sauvegarde du modèle entraîné avec joblib
model_filename_lr = '/content/sample_data/logistic_regression_model.joblib'
joblib.dump(clf_lr, model_filename_lr)
print(f"Modèle Logistic Regression sauvegardé sous le nom {model_filename_lr}")

# Sauvegarde du vectorizer
vectorizer_filename_lr = '/content/sample_data/tfidf_vectorizer_regretion.joblib'
joblib.dump(vectorizer, vectorizer_filename_lr)
print(f"Vectorizer Logistic Regression sauvegardé sous le nom {vectorizer_filename_lr}")

# Charger le modèle et le vectorizer pour faire des prédictions
loaded_model_lr = joblib.load(model_filename_lr)
loaded_vectorizer_lr = joblib.load(vectorizer_filename_lr)

# Exemple de prédiction sur un nouveau message
new_message = input("Entrer votre message: ")
new_message_tfidf = loaded_vectorizer_lr.transform([clean_text(new_message)])  # Passer le message sous forme de liste
prediction_lr = loaded_model_lr.predict(new_message_tfidf)

print(f"Prédiction pour le nouveau message (Logistic Regression): {prediction_lr[0]}")

Accuracy (Logistic Regression): 0.967713004484305
Classification Report (Logistic Regression):
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98       958
        spam       0.99      0.78      0.87       157

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

Modèle Logistic Regression sauvegardé sous le nom /content/sample_data/logistic_regression_model.joblib
Vectorizer Logistic Regression sauvegardé sous le nom /content/sample_data/tfidf_vectorizer_regretion.joblib
Entrer votre message: Nitady teny mety aho mba hisaorana anao amin'ity fofonaina ity. Mampanantena aho fa tsy horaisiko ho tsinontsinona ny fanampianao ary hotanterahiko ny fampanantenako. Nahafinaritra sy fitahiana foana ianao.
Prédiction pour le nouveau message (Logistic Regression): ham
